In [ ]:
from datetime import datetime, timedelta
import random

# PAYMENT MANAGEMENT SYSTEM
# This payment management script allows for the processing of payments, reminders, and penalties
# for policyholders who have purchased insurance products.
# A 5% penalty charge applied against the policyholder on their unpaid balance for the insurance product.

class Payment:
    def __init__(self, product, due_date, policyholder):
        self.product = product
        self.amount = product.price
        self.due_date = due_date
        self.policyholder = policyholder
        self.paid = False
        self.penalty = 0
        self.paid_amount = 0
        self.reminder_sent_date = None

    def process_payment(self):
        self.paid_amount = random.randint(int(self.amount * 0.5), self.amount)
        self.paid = True
        self.policyholder.add_payment(self)

    def send_reminder(self):
        if not self.paid or self.paid_amount < self.amount:
            self.policyholder.add_reminder(
                f"Reminder: Payment of ${self.amount} for {self.product.name} is due for {self.policyholder.name}"
            )
            self.reminder_sent_date = datetime.now()

    def apply_penalty(self):
        if not self.paid and datetime.now() > self.due_date and self.penalty == 0:
            self.penalty = random.randint(5, 30)
            self.policyholder.add_reminder(
                f"Penalty of ${self.penalty} applied to {self.policyholder.name} for late payment on {self.product.name}."
            )
        elif self.paid_amount < self.amount and self.reminder_sent_date and datetime.now() > self.reminder_sent_date + timedelta(days=30) and self.penalty == 0:
            diff = self.amount - self.paid_amount
            self.penalty = round(diff * 0.05, 2)
            self.policyholder.add_reminder(
                f"${self.penalty}% is 5% penalty charge applied against {self.policyholder.name} on unpaid balance for {self.product.name}."
            )

class Product:
    def __init__(self, name, min_price, max_price):
        self.name = name
        self.price = random.randint(min_price, max_price)

class Policyholder:
    def __init__(self, name):
        self.name = name
        self.payments = []
        self.penalty_amount = 0
        self.reminders = []

    def add_payment(self, payment):
        self.payments.append(payment)

    def add_reminder(self, reminder):
        self.reminders.append(reminder)

    def update_penalty_amount(self):
        self.penalty_amount = sum(p.penalty for p in self.payments)

    def apply_penalties(self):
        for payment in self.payments:
            payment.apply_penalty()
        self.update_penalty_amount()

    def __str__(self):
        info = "\n".join(
            [
                f"{p.product.name}: Price=${p.product.price}, Paid Amount=${p.paid_amount}, Paid={p.paid}, Due={p.due_date.strftime('%d-%m-%Y')}"
                for p in self.payments
            ]
        ) or "No payments."
        reminders = "\n".join(self.reminders) or "No reminders."
        return (
            f"Policyholder: {self.name}\n"
            f"Product paid for:\n{info}\n"
            f"Total Penalty Amount: ${self.penalty_amount}\n\n"
            f"{reminders}"
        )

# Example usage:
product1 = Product("Health Insurance", 150, 600)
product2 = Product("Car Insurance", 100, 700)
holder1 = Policyholder("Victor Ofili")
holder2 = Policyholder("Fego Ofili")
payment1 = Payment(product1, datetime.now() + timedelta(days=5), holder1)
payment2 = Payment(product2, datetime.now() + timedelta(days=5), holder2)
payment1.process_payment()
payment2.process_payment()
payment1.send_reminder()
payment2.send_reminder()
if payment1.reminder_sent_date:
    payment1.reminder_sent_date -= timedelta(days=31)
if payment2.reminder_sent_date:
    payment2.reminder_sent_date -= timedelta(days=31)
holder1.apply_penalties()
holder2.apply_penalties()
print("== Policyholder Accounts ==")
print()
print(holder1)
print()
print(holder2)


== Policyholder Accounts ==

Policyholder: Victor Ofili
Product paid for:
Health Insurance: Price=$391, Paid Amount=$220, Paid=True, Due=30-05-2025
Total Penalty Amount: $8.55

Reminder: Payment of $391 for Health Insurance is due for Victor Ofili
$8.55% is 5% penalty charge applied against Victor Ofili on unpaid balance for Health Insurance.

Policyholder: Fego Ofili
Product paid for:
Car Insurance: Price=$326, Paid Amount=$215, Paid=True, Due=30-05-2025
Total Penalty Amount: $5.55

Reminder: Payment of $326 for Car Insurance is due for Fego Ofili
$5.55% is 5% penalty charge applied against Fego Ofili on unpaid balance for Car Insurance.
